In [ ]:
##This script uses the normal CDK fingerprints and pfam counts
##but the callback reduceLrOnPlateau is used to try to
##optimise the changing of the learning rate automatically
##Early stopping was also implemented in case the loss function doesnt vary

In [1]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints


In [2]:
import numpy as np

In [3]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [4]:
def splittingTrainingDataIntoXAndY (filesData):##Receives a list in which element has as [0] the X and as [1] the Y
    xData=[]
    yData=[]
    for element in filesData:
        xData.append(element[0])
        yData.append(element[1])
    return xData, yData

In [5]:
filesData=[]
numberOfClusters=0
XFile=open("pfamDomains0and1ForAllClusters.txt")
YFile=open("fingerPrintsForIndividualClustersNOT478.txt")
for pfam, fingerprint in zip(XFile,YFile): ##Reads the two files at the same time and adds each thing to the same list
    pfam=pfam.strip("\n")
    fingerprint=fingerprint.strip("\n")
    filesData.append([convertStringIntoVector(pfam[11:]),convertStringIntoVector(fingerprint[11:])])
    numberOfClusters+=1
XFile.close()
YFile.close()

In [6]:
xData,yData = splittingTrainingDataIntoXAndY(filesData)##Now the list is splitted into X and Y data
xData=np.concatenate(xData,axis=0)
xData=xData.reshape(numberOfClusters,6159)
yData=np.concatenate(yData,axis=0)
yData=yData.reshape(numberOfClusters,306)
#The reshape puts the data as one row for each training example

In [7]:
import random
#Creates a list of Random Numbers as big as the number of Clusters we have
#Then we will select which ones we have for each subset
listOfRandomNumbers=random.sample(range(numberOfClusters), numberOfClusters)

In [8]:
xTestData = np.asarray([xData[i] for i in listOfRandomNumbers[:206]],dtype=int)
xTrainingData = np.asarray([xData[i] for i in listOfRandomNumbers[206:]],dtype=int)
yTestData = np.asarray([yData[i] for i in listOfRandomNumbers[:206]],dtype=int)
yTrainingData = np.asarray([yData[i]for i in listOfRandomNumbers[206:]],dtype=int)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import initializers
from keras.callbacks import LearningRateScheduler, CSVLogger, EarlyStopping, TensorBoard, ReduceLROnPlateau
lr=0.01
nn=[6159, 50, 306]

Using TensorFlow backend.


In [10]:
#### CALL BACKS
callback1 = ReduceLROnPlateau(monitor='loss', factor=0.4, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callback2 = EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, verbose=1, mode='min')

In [11]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2], epochs=120)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/120
825/825 [==============================] - 18s 22ms/step - loss: 0.2410 - binary_accuracy: 0.9061
Epoch 2/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1201 - binary_accuracy: 0.9526
Epoch 3/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1084 - binary_accuracy: 0.9562
Epoch 4/120
825/825 [==============================] - 15s 19ms/step - loss: 0.0997 - binary_accuracy: 0.9593
Epoch 5/120
825/825 [==============================] - 16s 19ms/step - loss: 0.0938 - binary_accuracy: 0.9616
Epoch 6/120
825/825 [==============================] - 16s 19ms/step - loss: 0.0878 - binary_accuracy: 0.9645
Epoch 7/120
825/825 [==============================] - 16s 19ms/step - loss: 0.0849 - binary_accuracy: 0.9661
Epoch 8/120
825/825 [==============================] - 16s 19ms/step - loss: 0.0820 - binary_accuracy: 0.9669


In [12]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [13]:
numberOfTrue13=0 ##Positions 13 and 299 in the fingerprint are around 50% of the time 1 and 0 so we are interested in how well the algorithm works for these positions
numberOfFalse13=0
numberOfTrue299=0
numberOfFalse299=0
for i in range (0, 206):
    if predictions[i][13]==yTestData[i][13]:
        numberOfTrue13 +=1
    else:
        numberOfFalse13 += 1
    if predictions [i][299]==yTestData[i][299]:
        numberOfTrue299 +=1
    else:
        numberOfFalse299+=1
print (numberOfTrue13)
print (numberOfFalse13)
print (numberOfTrue299)
print (numberOfFalse299)

147
59
151
55


In [14]:
def gaussianKernel (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [15]:
score1 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score1[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore1",score1)

In [16]:
xTestData = np.asarray([xData[i] for i in listOfRandomNumbers[206:412]],dtype=int)
xTrainingData = np.asarray([xData[i] for i in (listOfRandomNumbers[:206]+listOfRandomNumbers[412:])],dtype=int)
yTestData = np.asarray([yData[i] for i in listOfRandomNumbers[206:412]],dtype=int)
yTrainingData = np.asarray([yData[i]for i in (listOfRandomNumbers[:206]+listOfRandomNumbers[412:])],dtype=int)

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import initializers
from keras.callbacks import LearningRateScheduler, CSVLogger, EarlyStopping, TensorBoard, ReduceLROnPlateau
lr=0.01
nn=[6159, 50, 306]

In [18]:
#### CALL BACKS
callback1 = ReduceLROnPlateau(monitor='loss', factor=0.4, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callback2 = EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, verbose=1, mode='min')

In [19]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2], epochs=120)

Epoch 1/120
825/825 [==============================] - 17s 21ms/step - loss: 0.2413 - binary_accuracy: 0.8977
Epoch 2/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1229 - binary_accuracy: 0.9514
Epoch 3/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1112 - binary_accuracy: 0.9554
Epoch 4/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1024 - binary_accuracy: 0.9583
Epoch 5/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0958 - binary_accuracy: 0.9604
Epoch 6/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0906 - binary_accuracy: 0.9627
Epoch 7/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0852 - binary_accuracy: 0.9656
Epoch 8/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0814 - binary_accuracy: 0.9669
Epoch 9/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0785 - binary_accuracy: 0.9686
Epoch 10/1

In [20]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [21]:
numberOfTrue13=0 ##Positions 13 and 299 in the fingerprint are around 50% of the time 1 and 0 so we are interested in how well the algorithm works for these positions
numberOfFalse13=0
numberOfTrue299=0
numberOfFalse299=0
for i in range (0, 206):
    if predictions[i][13]==yTestData[i][13]:
        numberOfTrue13 +=1
    else:
        numberOfFalse13 += 1
    if predictions [i][299]==yTestData[i][299]:
        numberOfTrue299 +=1
    else:
        numberOfFalse299+=1
print (numberOfTrue13)
print (numberOfFalse13)
print (numberOfTrue299)
print (numberOfFalse299)

146
60
153
53


In [22]:
def gaussianKernel (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [23]:
score2 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score2[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore2",score2)

In [24]:
xTestData = np.asarray([xData[i] for i in listOfRandomNumbers[412:618]],dtype=int)
xTrainingData = np.asarray([xData[i] for i in (listOfRandomNumbers[:412]+listOfRandomNumbers[618:])],dtype=int)
yTestData = np.asarray([yData[i] for i in listOfRandomNumbers[412:618]],dtype=int)
yTrainingData = np.asarray([yData[i]for i in (listOfRandomNumbers[:412]+listOfRandomNumbers[618:])],dtype=int)

In [25]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import initializers
from keras.callbacks import LearningRateScheduler, CSVLogger, EarlyStopping, TensorBoard, ReduceLROnPlateau
lr=0.01
nn=[6159, 50, 306]

In [26]:
#### CALL BACKS
callback1 = ReduceLROnPlateau(monitor='loss', factor=0.4, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callback2 = EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, verbose=1, mode='min')

In [27]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2], epochs=120)

Epoch 1/120
825/825 [==============================] - 17s 21ms/step - loss: 0.2200 - binary_accuracy: 0.9009
Epoch 2/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1258 - binary_accuracy: 0.9513
Epoch 3/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1122 - binary_accuracy: 0.9545
Epoch 4/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1021 - binary_accuracy: 0.9583
Epoch 5/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0968 - binary_accuracy: 0.9608
Epoch 6/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0923 - binary_accuracy: 0.9623
Epoch 7/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0893 - binary_accuracy: 0.9637
Epoch 8/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0859 - binary_accuracy: 0.9652
Epoch 9/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0840 - binary_accuracy: 0.9662
Epoch 10/1

In [28]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [29]:
numberOfTrue13=0 ##Positions 13 and 299 in the fingerprint are around 50% of the time 1 and 0 so we are interested in how well the algorithm works for these positions
numberOfFalse13=0
numberOfTrue299=0
numberOfFalse299=0
for i in range (0, 206):
    if predictions[i][13]==yTestData[i][13]:
        numberOfTrue13 +=1
    else:
        numberOfFalse13 += 1
    if predictions [i][299]==yTestData[i][299]:
        numberOfTrue299 +=1
    else:
        numberOfFalse299+=1
print (numberOfTrue13)
print (numberOfFalse13)
print (numberOfTrue299)
print (numberOfFalse299)

154
52
155
51


In [30]:
def gaussianKernel (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [31]:
score3 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score3[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore3",score3)

In [32]:
xTestData = np.asarray([xData[i] for i in listOfRandomNumbers[618:824]],dtype=int)
xTrainingData = np.asarray([xData[i] for i in (listOfRandomNumbers[:618]+listOfRandomNumbers[824:])],dtype=int)
yTestData = np.asarray([yData[i] for i in listOfRandomNumbers[618:824]],dtype=int)
yTrainingData = np.asarray([yData[i]for i in (listOfRandomNumbers[:618]+listOfRandomNumbers[824:])],dtype=int)

In [33]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import initializers
from keras.callbacks import LearningRateScheduler, CSVLogger, EarlyStopping, TensorBoard, ReduceLROnPlateau
lr=0.01
nn=[6159, 50, 306]

In [34]:
#### CALL BACKS
callback1 = ReduceLROnPlateau(monitor='loss', factor=0.4, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callback2 = EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, verbose=1, mode='min')

In [35]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2], epochs=120)

Epoch 1/120
825/825 [==============================] - 17s 21ms/step - loss: 0.2208 - binary_accuracy: 0.9019
Epoch 2/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1222 - binary_accuracy: 0.9516
Epoch 3/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1114 - binary_accuracy: 0.9536
Epoch 4/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1031 - binary_accuracy: 0.9574
Epoch 5/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0980 - binary_accuracy: 0.9595
Epoch 6/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0940 - binary_accuracy: 0.9620
Epoch 7/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0876 - binary_accuracy: 0.9648
Epoch 8/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0832 - binary_accuracy: 0.9659
Epoch 9/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0811 - binary_accuracy: 0.9670
Epoch 10/1

In [36]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [37]:
numberOfTrue13=0 ##Positions 13 and 299 in the fingerprint are around 50% of the time 1 and 0 so we are interested in how well the algorithm works for these positions
numberOfFalse13=0
numberOfTrue299=0
numberOfFalse299=0
for i in range (0, 206):
    if predictions[i][13]==yTestData[i][13]:
        numberOfTrue13 +=1
    else:
        numberOfFalse13 += 1
    if predictions [i][299]==yTestData[i][299]:
        numberOfTrue299 +=1
    else:
        numberOfFalse299+=1
print (numberOfTrue13)
print (numberOfFalse13)
print (numberOfTrue299)
print (numberOfFalse299)

148
58
147
59


In [38]:
def gaussianKernel (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [39]:
score4 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score4[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore4",score4)

In [40]:
xTestData = np.asarray([xData[i] for i in listOfRandomNumbers[825:]],dtype=int)
xTrainingData = np.asarray([xData[i] for i in listOfRandomNumbers[:825]],dtype=int)
yTestData = np.asarray([yData[i] for i in listOfRandomNumbers[825:]],dtype=int)
yTrainingData = np.asarray([yData[i]for i in listOfRandomNumbers[:825]],dtype=int)

In [41]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import initializers
from keras.callbacks import LearningRateScheduler, CSVLogger, EarlyStopping, TensorBoard, ReduceLROnPlateau
lr=0.01
nn=[6159, 50, 306]

In [42]:
#### CALL BACKS
callback1 = ReduceLROnPlateau(monitor='loss', factor=0.4, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callback2 = EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, verbose=1, mode='min')

In [43]:
model=Sequential()
model.add(Dense(nn[0], input_dim=6159, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=lr), metrics = ['binary_accuracy'])
model.fit(xTrainingData, yTrainingData,callbacks=[callback1,callback2], epochs=120)

Epoch 1/120
825/825 [==============================] - 18s 21ms/step - loss: 0.2291 - binary_accuracy: 0.8995
Epoch 2/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1216 - binary_accuracy: 0.9532
Epoch 3/120
825/825 [==============================] - 15s 18ms/step - loss: 0.1038 - binary_accuracy: 0.9580
Epoch 4/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0957 - binary_accuracy: 0.9610
Epoch 5/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0921 - binary_accuracy: 0.9620
Epoch 6/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0866 - binary_accuracy: 0.9647
Epoch 7/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0820 - binary_accuracy: 0.9665
Epoch 8/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0772 - binary_accuracy: 0.9684
Epoch 9/120
825/825 [==============================] - 15s 18ms/step - loss: 0.0760 - binary_accuracy: 0.9691
Epoch 10/1

In [46]:
predictions = model.predict(xTestData) ##Rounds up and down every prediction to 1 and 0
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [48]:
numberOfTrue13=0 ##Positions 13 and 299 in the fingerprint are around 50% of the time 1 and 0 so we are interested in how well the algorithm works for these positions
numberOfFalse13=0
numberOfTrue299=0
numberOfFalse299=0
for i in range (0, 206):
    if predictions[i][13]==yTestData[i][13]:
        numberOfTrue13 +=1
    else:
        numberOfFalse13 += 1
    if predictions [i][299]==yTestData[i][299]:
        numberOfTrue299 +=1
    else:
        numberOfFalse299+=1
print (numberOfTrue13)
print (numberOfFalse13)
print (numberOfTrue299)
print (numberOfFalse299)

153
53
157
49


In [49]:
def gaussianKernel (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [50]:
score5 = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook
for i in range (0, 206):
    for j in range (0, 206):
        score5[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kerasScore5",score5)